In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torch.optim.lr_scheduler as lr_scheduler
from scipy.io import savemat
import random

def h_broadcast(t, r, wavelength=0.01, device="cuda" if torch.cuda.is_available() else "cpu"):
    eta = 120 * torch.pi
    k0 = 2 * torch.pi / wavelength

    # polarization vector e (y-direction)
    e = torch.tensor([0.0, 1.0, 0.0], dtype=torch.float32, device=device).view(1, 3, 1)

    # t: (100, 3), r: (50, 3)
    # 확장해서 (50, 100, 3)
    t_exp = t[None, :, :].expand(r.shape[0], -1, -1)   # (50, 100, 3)
    r_exp = r[:, None, :].expand(-1, t.shape[0], -1)   # (50, 100, 3)

    diff = r_exp - t_exp                      # (50, 100, 3)
    norm = torch.norm(diff, dim=2, keepdim=True)       # (50, 100, 1)
    norm_squeezed = norm.squeeze(-1)          # (50, 100)

    phase_real = torch.cos(-k0 * norm_squeezed)  # (50, 100)
    phase_imag = torch.sin(-k0 * norm_squeezed)  # (50, 100)
    phase = torch.complex(phase_real, phase_imag)  # (50, 100)

    # Projection matrix
    diff_u = diff.unsqueeze(3)  # (50, 100, 3, 1)
    outer = diff_u @ diff_u.transpose(2, 3) / (norm ** 2).unsqueeze(3)  # (50, 100, 3, 3)
    I = torch.eye(3, device=device).view(1, 1, 3, 3).expand(r.shape[0], t.shape[0], 3, 3)
    proj = I - outer  # (50, 100, 3, 3)

    # Polarization response
    e_exp = e.view(1, 1, 3, 1).expand(r.shape[0], t.shape[0], 3, 1)
    polarization = (e_exp.transpose(2, 3) @ (proj @ e_exp)).squeeze(-1).squeeze(-1)  # (50, 100)

    coeff = -1j * eta / (2 * wavelength * norm_squeezed)  # (50, 100)
    return (coeff * phase * polarization)  # shape: (50, 100)


loss_history = {"LR1": [], "LR2": [], "LR3": [], "LR4": [], "LR5": [], "L_ic_op": [], "L_ic1": [], "L_ic2": []}

class ResidualBlock(nn.Module):
    def __init__(self, in_features, out_features, activation=nn.SiLU()):
        super(ResidualBlock, self).__init__()
        self.activation = activation
        self.fc1 = nn.Linear(in_features, out_features)
        self.fc2 = nn.Linear(out_features, out_features)
        if in_features != out_features:
            self.skip = nn.Linear(in_features, out_features)
        else:
            self.skip = nn.Identity()

    def forward(self, x):
        residual = self.skip(x)
        out = self.activation(self.fc1(x))
        out = self.activation(self.fc2(out))
        out = out + residual
        return out

In [2]:
class Complex_Model(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, num_blocks, act=nn.SiLU(), out_act=None):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.blocks = nn.ModuleList([ResidualBlock(hidden_dim, hidden_dim, act) for _ in range(num_blocks)])
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.act = act
        self.out_act = out_act

    def forward(self, x):
        x = self.act(self.input_layer(x))
        for block in self.blocks:
            x = block(x)
        x = self.output_layer(x)

        # 출력 차원 처리
        batch_size = x.shape[0]

        # 홀수이면 그대로 반환
        if x.shape[1] % 2 != 0:
            return x

        # 짝수이면 복소수로 변환
        half_dim = x.shape[1] // 2  # 절반 크기
        x = x.view(batch_size, half_dim, 2)

        # 복소수 변환
        real_part = x[:, :, 0].unsqueeze(2)  # 실수부
        imag_part = x[:, :, 1].unsqueeze(2)  # 허수부
        complex_output = torch.complex(real_part, imag_part)

        return complex_output

class Real_Model(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, num_blocks, act=nn.SiLU(), out_act=None):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.blocks = nn.ModuleList([ResidualBlock(hidden_dim, hidden_dim, act) for _ in range(num_blocks)])
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.act = act
        self.out_act = out_act

    def forward(self, x):
        x = self.act(self.input_layer(x))
        for block in self.blocks:
            x = block(x)
        x = self.output_layer(x)
        return x


In [ ]:
mse = nn.MSELoss()

Signal = Complex_Model(input_dim=9, output_dim=4, hidden_dim=256, num_blocks=2)
mp_integral = Complex_Model(input_dim=9, output_dim=8, hidden_dim=256, num_blocks=2)
objective_integral = Real_Model(input_dim=9, output_dim=1, hidden_dim=256, num_blocks=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Signal.to(device)
mp_integral.to(device)
objective_integral.to(device)

optimizer1 = optim.Adam(Signal.parameters(), lr=1e-3)
optimizer2 = optim.Adam(mp_integral.parameters(), lr=1e-3)
optimizer3 = optim.Adam(objective_integral.parameters(), lr=1e-3)

N = 10

# SINR Constraint
gamma_db_1 = 5  # dB
gamma_db_2 = 5  # dB
gamma_p_1 = 10**(gamma_db_1 / 10)
gamma_p_2 = 10**(gamma_db_2 / 10)

sigma_sq = 1

# Number of Users
P = 2

gamma_list = torch.tensor([gamma_p_1, gamma_p_2]) 

C_list = []

for p in range(P):
    c = torch.ones(P) / sigma_sq
    c[p] = -1.0 / (gamma_list[p] * sigma_sq)
    C_p = torch.diag(c).to(device)
    C_list.append(C_p)
    
# C matrix for Constraint
C_1, C_2 = C_list

iterations = 100000
num_tx = 500
batch_size = 50
Total = []

for epoch in range(iterations):

    # 손실항 초기화
    L_ic1 = L_ic2 = L_ic_op = LR1 = LR2 = LR3 = LR4 = LR5 = 0.0
    
    # Gradient-Zero Condition
    optimizer1.zero_grad()
    optimizer2.zero_grad()    
    optimizer3.zero_grad()
    
    # 안테나 생성
    tx = torch.tensor(np.random.uniform(-10, 10, num_tx), dtype=torch.float32, device=device).unsqueeze(1)   # (500, 1)
    ty = torch.zeros((num_tx, 1), dtype=torch.float32, device=device)                                        # (500, 1)
    tz = torch.zeros((num_tx, 1), dtype=torch.float32, device=device)                                        # (500, 1)
    t = torch.cat([tx, ty, tz], dim=1)  # shape: (500, 3)
    
    # 유저1 생성
    u1n = np.zeros((batch_size, 3))
    u1n[:, 0] = np.random.uniform(-1, 1, batch_size)
    u1n[:, 2] = 30
    u1 = torch.from_numpy(u1n).float().to(device)

    # 유저2 생성
    u2n = np.zeros((batch_size, 3))
    u2n[:, 0] = np.random.uniform(-1, 1, batch_size)
    u2n[:, 2] = 30
    u2 = torch.from_numpy(u2n).float().to(device)

    u1_exp = u1[:, None, :].expand(-1, num_tx, -1)  
    u2_exp = u2[:, None, :].expand(-1, num_tx, -1)  

    t_exp = t[None, :, :].expand(batch_size, -1, -1)     

    inputs = torch.cat([u1_exp, u2_exp, t_exp], dim=-1)

    H1 = h_broadcast(t, u1)  # H1 shape: (50, 500)
    H2 = h_broadcast(t, u2)

    Signal_output = Signal(inputs.view(-1, 9)).view(batch_size, num_tx, 2)
    mp_integral_output = mp_integral(inputs.view(-1, 9)).view(batch_size, num_tx, 2, 2)
    objective_integral_output = objective_integral(inputs.view(-1, 9)).view(batch_size, num_tx, 1)

    q1, q2 = Signal[:, :, 0], Signal_output[:, :, 1]


    LR2 = F.mse_loss((torch.conj(h1) * q1).real, d2_mp[:, :, 0].real) + F.mse_loss((torch.conj(h1) * q1).imag, d2_mp[:, :, 0].imag)
    LR3 = F.mse_loss((torch.conj(h1) * q2).real, d2_mp[:, :, 1].real) + F.mse_loss((torch.conj(h1) * q2).imag, d2_mp[:, :, 1].imag)
    LR4 = F.mse_loss((torch.conj(h2) * q1).real, d2_mp[:, :, 2].real) + F.mse_loss((torch.conj(h2) * q1).imag, d2_mp[:, :, 2].imag)
    LR5 = F.mse_loss((torch.conj(h2) * q2).real, d2_mp[:, :, 3].real) + F.mse_loss((torch.conj(h2) * q2).imag, d2_mp[:, :, 3].imag)



    # 적분 항들
    t_pos = torch.tensor([[10.0, 0.0, 30.0]], device=device).expand(batch_size, -1)
    t_neg = torch.tensor([[-10.0, 0.0, 30.0]], device=device).expand(batch_size, -1)

    input_pos = torch.cat([u1, u2, t_pos], dim=1)  # (B, 9)
    input_neg = torch.cat([u1, u2, t_neg], dim=1)

    val_pos = mp_integral(input_pos).view(batch_size,2,2)
    val_neg = mp_integral(input_neg).view(batch_size,2,2)

    int_mp = val_pos - val_neg

    C_1 = C_1.to(dtype=torch.complex64, device=device)
    C_2 = C_2.to(dtype=torch.complex64, device=device)

    term1 = torch.einsum('bi,ij,bj->b', int_mp[:, 0].conj(), C_1, int_mp[:, 0])
    term2 = torch.einsum('bi,ij,bj->b', int_mp[:, 1].conj(), C_2, int_mp[:, 1])

    # 부등호 제약 조건
    L_ic1 = torch.relu(term1.real + 1).sum()
    L_ic2 = torch.relu(term2.real + 1).sum()


    # 목적 함수
    pos_obj = objective_integral(input_pos)
    neg_obj = objective_integral(input_neg)
    
    int_obj = pos_obj - neg_obj
    L_ic_op = int_obj.sum()



    for key, value in zip(loss_history.keys(), [LR1, LR2, LR3, LR4, LR5, L_ic_op, L_ic1, L_ic2]):
        loss_history[key].append(value.item())

    Total_Loss = LR1 + 3*LR2 + 3*LR3 + 3*LR4 + 3*LR5 + L_ic1 + L_ic2 + 1e-8 * L_ic_op

    Total_Loss.backward()
    optimizer1.step()
    optimizer2.step()
    optimizer3.step()

    Total.append(Total_Loss.item())

    with torch.no_grad():
        print(epoch, f"Total Loss: {Total_Loss.item():.6f}")

    if epoch % 100 == 0:
        plt.figure(figsize=(8, 6))
        plt.plot(Total, label="Total Loss", color=np.random.rand(3,))
        plt.xlabel("Epochs")
        plt.ylabel("Log Loss Value")
        plt.title("Loss during Training")
        plt.legend()
        plt.grid()
        plt.show()

    if epoch % 100 == 0:
        for key, values in loss_history.items():
            plt.figure(figsize=(8, 6))
            if key != "L_ic_op":
                values = np.log10(np.maximum(np.abs(values), 1e-16))
            plt.plot(values, label=key, color=np.random.rand(3,))
            plt.xlabel("Epochs")
            plt.ylabel("Log Loss Value" if key != "L_ic_op" else "Loss Value")
            plt.title(f"{key} During Training")
            plt.legend()
            plt.grid()
            plt.show()

for key, values in loss_history.items():
    plt.figure(figsize=(8, 5))
    if key != "L_ic_op":
        values = np.log10(np.maximum(np.abs(values), 1e-16))
    plt.plot(values, label=key, color=np.random.rand(3,))
    plt.xlabel("Epochs")
    plt.ylabel("Log Loss Value" if key != "L_ic_op" else "Loss Value")
    plt.title(f"{key} During Training")
    plt.legend()
    plt.grid()
    plt.savefig(f"{key.replace(' ', '_')}_history.png")
    plt.show()


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument tensors in method wrapper_CUDA_cat)

True
NVIDIA GeForce RTX 4080
